In [2]:
from pandas import Series, DataFrame
import pandas as pd
from pandas.tseries.offsets import Day, MonthEnd
import numpy as np
import sys

import matplotlib.pyplot as plt

from scipy.stats import rankdata
from scipy.stats import stats
from scipy.optimize import minimize

%matplotlib inline

try:
  from pandas_datareader import data
  import pandas_datareader.data as web
except:
  !pip install pip pandas-datareader
  
  from pandas_datareader import data
  import pandas_datareader.data as web
 

All Weather Portfolio

In [110]:
# 일단 회사채는 받아오긴하는데 아래는 안썼어요

equity = web.get_data_yahoo('SPY', start ='2008-01-29', end = '2020-11-30')['Adj Close']
equity2 = web.get_data_yahoo('VXF', start ='2008-01-29', end = '2020-11-30')['Adj Close']
equity3 = web.get_data_yahoo('VB', start ='2008-01-29', end = '2020-11-30')['Adj Close']
bond1 = web.get_data_yahoo('BND', start ='2008-01-29', end = '2020-11-30')['Adj Close']
bond2 = web.get_data_yahoo('EDV', start ='2008-01-29', end = '2020-11-30')['Adj Close']
comm1 = web.get_data_yahoo('DBB', start ='2008-01-29', end = '2020-11-30')['Adj Close']
comm2 = web.get_data_yahoo('DBA', start ='2008-01-29', end = '2020-11-30')['Adj Close']
  

In [111]:
#Equity : 주식, infl : 물가연동채권(inflation-linked), corp : 회사채, bond : 국채, emd : 이머징국가채권, comm : 원자재

equity = equity.pct_change().dropna()*100  + 1
equity2 = equity2.pct_change().dropna()*100  + 1
equity3 = equity3.pct_change().dropna()*100  + 1
bond1 = bond1.pct_change().dropna()*100  + 1
bond2 = bond2.pct_change().dropna()*100  + 1
comm1 = comm1.pct_change().dropna()*100  + 1
comm2 = comm2.pct_change().dropna()*100  + 1


In [115]:
#수익률 DF 하나로 합치는거
port = pd.concat([equity, bond2, comm1, comm2], axis =1 )
port.columns = ['equity','bond2' ,'comm1','comm2']
asset_num = len(port.columns)

#Covariance Matrix
covmat = port.cov() 

covmat

,equity,bond2,comm1,comm2
equity,1.747824,-0.789153,0.786921,0.496107
bond2,-0.789153,2.062939,-0.513399,-0.279839
comm1,0.786921,-0.513399,2.139938,0.680274
comm2,0.496107,-0.279839,0.680274,1.166609


In [116]:
def RiskParity_objective(x):
    variance = x.T @ covmat @ x
    sigma = variance ** 0.5
    mrc = 1 / sigma * (covmat @ x)
    rc = x * mrc
    rc = rc / rc.sum()

    a = pd.DataFrame(index=range(len(x)), columns=range(len(x)))
    for i in range(0, len(x)):
        for j in range(0, len(x)):
            a[i][j] = rc[i] - rc[j]

    sum_risk_diffs_squared = np.sum(np.square(np.ravel(a)))
    return (sum_risk_diffs_squared)


def weight_sum_constraint(x):
    return (x.sum() - 1.0)


def weight_longonly(x):
    return (x)


def RiskParity(covmat):
    x0 = np.repeat(1 / covmat.shape[1], covmat.shape[1])
    con1 = {'type': 'eq', 'fun': weight_sum_constraint}
    con2 = {'type': 'ineq', 'fun': weight_longonly}
 

    constraint = ([con1, con2])

    options = {'ftol': 1e-20, }

    result = minimize(fun=RiskParity_objective,
                      x0=x0,
                      method='SLSQP',
                      constraints=constraint,
                      options=options)
    return (result.x)


wt_erc = RiskParity(covmat)

wt_erc

array([0.2451816 , 0.32891352, 0.18296546, 0.24293941])

In [31]:
x = np.repeat(1 / covmat.shape[1], covmat.shape[1])

var = x.T @ covmat @ x.T

var

0.9775741590314317

In [74]:
def GMV_objective(x):
  variance = x.T @ covmat @ x
  return (variance)

def weight_sum_constraint(x):
    return (x.sum() - 1.0)

def weight_least(x):
    return(x[0] - 0.1)
def weight_longonly(x):
    return (x)

def GMV(covmat):
  x0 = np.repeat(1 / covmat.shape[1], covmat.shape[1])
  con1 = {'type': 'eq', 'fun': weight_sum_constraint}
  con2 = {'type': 'ineq', 'fun': weight_longonly} 
  #con3 = {'type': 'ineq', 'fun' :weight_least}

  constraint = ([con1, con2])
  options = {'ftol': 1e-20, }
  result = minimize(fun=GMV_objective, x0=x0,
                      method='SLSQP',
                      constraints=constraint,
                      options=options)
    
  return (result.x)


gmv_ret = GMV(covmat)

gmv_ret


array([0.21868435, 0.31110039, 0.19247394, 0.27774132])